# Лабораторная работа 4.

## Решение СЛАУ с помощью GMRes и BiCGstab методов.

#### Вариант 2

In [77]:
import numpy as np
import random

In [654]:
def print_r(x):
    print(np.round(x, 6))


def get_sin_and_cos(a, b):
    c = None
    s = None
    if (b == 0):
        c = 1
        s = 0
    else:
        if (abs(b) > abs(a)):
            tau = a / b
            s = 1 / np.sqrt(1 + tau ** 2)
            c = s * tau
        else:
            tau = b / a
            c = 1 / np.sqrt(1 + tau ** 2)
            s = c * tau
    return s ,c


def givens_apply_rotate(a, b, c, s):
    return c * a + s * b, - s * a + c * b


def gauss_solve(a, b):
    n = b.shape[0]
    x = np.empty(n)
    x[n - 1] = -b[n - 1] / a[n - 1][n - 1]
    diag = n - 2
    for i in range(n - 2, -1, -1):
        s = b[i]
        for j in range(n - 1, diag, -1):
            s -= a[i][j] * x[j]
        x[i] = s / a[i][diag]
        diag -= 1
    return x


def gmres_solve(A, b, eps=10e-5):
    n = A.shape[0]
    x0 = np.zeros(n)
    r0 = b - A @ x0
    beta = np.linalg.norm(r0)
    v0 = r0 * (1 / beta)
    g = np.zeros(n)
    g[0] = beta
    p = n
    H = np.zeros((n + 1, n))
    V = [v0]
    c = np.zeros(n)
    s = np.zeros(n)
    for j in range(n):
        w = A @ V[j]
        for i in range(j + 1):
            H[i, j] = np.dot(w, V[i])
            w = w - H[i, j] * V[i]
        H[j + 1, j] = np.linalg.norm(w)
        V.append(w * (1 / H[j + 1, j]))
        if (abs(H[j + 1, j]) < eps):
            p = j + 1
            break
        h_r = H[:j + 2, j]
        for i in range(j):
            h_r[i], h_r[i + 1] = givens_apply_rotate(h_r[i], h_r[i + 1],  c[i], s[i])
            g[i], g[i + 1] = givens_apply_rotate(g[i], g[i + 1],  c[i], s[i])
        s[j], c[j] = get_sin_and_cos(H[j, j], H[j + 1, j])
        h_r[j], h_r[j + 1] = givens_apply_rotate(h_r[j], h_r[j + 1],  c[j], s[j])
        H[:j + 2, j] = h_r
        g[j], g[j + 1] = givens_apply_rotate(g[j], g[j + 1],  c[j], s[j])
        if (abs(g[j + 1]) < eps):
            p = j + 1
            break
    y = np.linalg.solve(H[:p, :p], g[:p])
    x = x0
    print_r(y)
    for i in range(p):
        x += y[i] * V[i]
    print(x)


In [655]:
N = 5
A = np.zeros((N, N))
x = np.zeros(N)
for i in range(N):
    for j in range(N):
        A[i, j] = float(random.randint(-10, 10))
    x[i] = float(random.randint(-10, 10))

b = A @ x
print_r(A)
print_r(x)
print_r(b)




[[  9.  -2.   1.   3.   9.]
 [  2.  10.   6.  -4.  10.]
 [ -8.   8.  -9.  -1.   0.]
 [ -2.  -9.  -5.  -3.   3.]
 [ -1. -10.   6.  -8.   0.]]
[  3.   1.  -5. -10.  -3.]
[-37.  -4.  39.  31.  37.]


In [653]:
gmres_solve(A, b)

[[-19.675358   0.         0.         0.         0.      ]
 [  0.         0.         0.         0.         0.      ]
 [  0.         0.         0.         0.         0.      ]
 [  0.         0.         0.         0.         0.      ]
 [  0.         0.         0.         0.         0.      ]
 [  0.         0.         0.         0.         0.      ]]
[[-19.675358   2.572737   0.         0.         0.      ]
 [  0.         8.101899   0.         0.         0.      ]
 [  0.         0.         0.         0.         0.      ]
 [  0.         0.         0.         0.         0.      ]
 [  0.         0.         0.         0.         0.      ]
 [  0.         0.         0.         0.         0.      ]]
[[-19.675358   2.572737   2.217446   0.         0.      ]
 [  0.         8.101899   8.472058   0.         0.      ]
 [  0.         0.        -4.16666    0.         0.      ]
 [  0.         0.         0.         0.         0.      ]
 [  0.         0.         0.         0.         0.      ]
 [  0.      